<a href="https://colab.research.google.com/github/RafaelCaballero/Julio25/blob/main/code/25bayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introducción a la ciencia de datos con Python
Rafa Caballero


## Naïve Bayes

Tenemos datos sobre mails y sobre si son o no SPAM (obtenidos de [aquí](https://www.codingninjas.com/codestudio/library/bernoulli-naive-bayes) )

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.naive_bayes import BernoulliNB
from sklearn.feature_extraction.text import CountVectorizer
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt



#### Carga y preprocesado
path = "https://raw.githubusercontent.com/amankharwal/SMS-Spam-Detection/master/spam.csv"
#path = "c:/hlocal/tdm/movimiento.csv"
df = pd.read_csv(path,encoding="latin1").drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], axis=1)
df

OSError: 'seaborn-whitegrid' is not a valid package style, path of style file, URL of style file, or library style name (library styles are listed in `style.available`)

**Ejercicio** ¿cuántos valores diferentes tiene la columna class?

Crear una nueva columna `label`que valga 1 si `class`es 'spam' y 0 en otro caso

In [ ]:
df["label"]=0
df.loc[df["class"]=="spam","label"] =1

Ahora quitamos las palabras comunes (stopwords), signos de puntuación, etc. Para ello utilizamos la librería `spacy`

In [ ]:
import spacy
nlp = spacy.blank('en')
stopwords = nlp.Defaults.stop_words

l = [" ".join([token.text.upper() for token in nlp(doc) if not token.is_stop and  token.is_alpha]) for doc in df.message]

df["limpio"] = l
df.sample(20)

Tenemos que convertir las palabras en columnas. El método `CountVectorizer` se encarga de ello. Importante: no se trata de reemplazar cada palabra de la frase por un número, sino de crear tantas columnas como palabras haya en total

In [ ]:
frases =df["limpio"].values
y = df["label"].values
# creating count vectorizer object
cv = CountVectorizer()
#tranforming values
X = cv.fit_transform(frases)
v= X.toarray()
#printing sparse matrix
print(v.shape)
v

Casi parece que solo hay 0s. Esto es porque es una matriz *dispersa*, es decir, llena de huecos (representados aquí por un 0). Pero sí que hay valores

In [ ]:
(v[1]!=0).sum()

In [ ]:
palabras = cv.get_feature_names_out()
print(len(palabras))
palabras[90:100]

Ahora probamos los distintos métodos basados en Naïve Bayes

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import classification_report
from sklearn.metrics import cohen_kappa_score
import matplotlib.pyplot as plt
def evaluar(y_test,y_pred):
  k =  cohen_kappa_score(y_test,y_pred)
  print("kappa ",k)
  cm = confusion_matrix(y_test, y_pred, labels=modelo.classes_)
  disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                                display_labels=modelo.classes_)
  disp.plot()

  plt.show()

  print(classification_report(y_test, y_pred))
  return k,cm

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# 2 preparar train y test
test = 0.4
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= test)

# 3 método y entrenamiento
metodo = BernoulliNB()
modelo = metodo.fit(X_train,y_train)

# 4 evaluar
y_pred = modelo.predict(X_test)
evaluar(y_test,y_pred)

**Pregunta** En el caso del SPAM, ¿Es mejor tener una precisión alta y  un recall un poco más bajo o al revés?

**Ejercicio** Queremos probar a obtener el coeficiente kappa con `BernoulliNB`con `RandomOverSampler` (repitiendo los valores menos frecuentes, los del Spam) y validación cruzada

Buscar al final del notebook Pipelines, copiar, pegar y hacer los cambios que correspondan

**Ejercicio** probar ahora con regresión logística

**Ejercicio** Probar con SMOTE ¿puedes explicar el resultado?

**Ejercicio** Probar con otros métodos SVM, RandomForest...